In [273]:
import pandas as pd
import numpy as np
import calendar

from bokeh.charts import output_notebook, Scatter, Bar, show, output_file, Line, BoxPlot, Scatter
from bokeh.plotting import figure
from bokeh.layouts import row, column, gridplot
from bokeh.charts import output_notebook, Scatter, Bar, show, output_file, Line, BoxPlot, Scatter
from bokeh.io import hplot

output_notebook() 

Loading BokehJS ...

In [274]:
INPUT="data/device_failure.csv" 
dataset = pd.read_csv(INPUT,index_col=[0,1],parse_dates=[0])

## per device and time model

 - Set up first model:
     - windowing group by
     - per device test/train split strategy
 - Precision/recall, ROC
 - Calibration
 - feature engineering
     - last n values ?
 - Test other models
 - TPOT

## Build Training set

In [53]:
# fitering methods
suspicious_positives = set(["S1F0GPFZ", "S1F136J0", "W1F0KCP2", "W1F0M35B", "W1F11ZG9"])
def filter_devices(df):
    return df.filter(axis="index",items=(set(df.index) - suspicious_positives)) 

In [54]:
# feature preprocessing
def build_deriv(df,c,n=1):
    def per_device(per_device):
        clean_index = per_device.reset_index(level=1,drop=True)
        resampled=clean_index.resample('1D').pad()
        
        raw_diff = np.diff(resampled,n=n)
        #fill the series start with zeros
        while (len(raw_diff) < len(resampled)):
            raw_diff = np.insert(raw_diff,0,0)
        d = pd.Series(data=raw_diff,index = resampled.index )
        #print d
        return d.dropna()[d>0]
    some_d= df[c].groupby(level="device").apply(per_device)
    return some_d.swaplevel()

In [55]:
from scipy.fftpack import fft
#fft_df = feature_dset[[attribute]].copy()

def to_fft(df):
    try:
        resampled =  df.resample("1D",level="date").mean().fillna(method='pad')
    except:
        #if we cannot resample, this (usually) means that we are using a rolling aggregation, outputing
        #an nd.array rather than a df. the good news is, in this case I shoudl already have resampled.
        resampled = df.copy()
    n = len(resampled)
    return np.abs(fft(resampled))[n//2:]

def fft_line(df):
    print df
    return df.groupby(level="device",sort=True).transform(to_fft)

def peaks(line):
    sorted_by_used = sorted(enumerate(line),key = lambda  t: t[1], reverse=True )
    boundaries = set()
    peaks = []
    for i,value in sorted_by_used:
        if i not in boundaries:
            peaks.append((i,value))
        # in any case, i neighbors cannot be peaks now.
        boundaries.add(i+1)
        boundaries.add(i-1)
    return peaks
            
    
def fft_peak(df,p=0,index_no_value=True):
    fft = to_fft(df)
    #fft = fft_line(df)
    all_peaks = peaks(fft.tolist())
    if (len(all_peaks) > p):
        return all_peaks[p][0 if index_no_value else 1]
    else:
        return 0

In [56]:
def pre_filter(df):
    res = df.copy()
    del res["attribute1"]
    del res["attribute3"]
    #del res["attribute5"]
    dt_list = ["attribute2"]#,"attribute8"]
    for c in dt_list:
        deriv = build_deriv(res,c)
        res["dt_%s" % c] = deriv
        res["dt2_%s" % c] =  build_deriv(res,c,2)
    return res.fillna(0)

def post_filter(df):
    res = df.copy()
    res = filter_devices(res)
    for col in res.columns:
        if "min" in col:
            del res[col]
        if "std" in col:
            del res[col]
    return res

In [330]:
pre_dataset = pre_filter(dataset)
#print feature_set.columns

features = [f for f in pre_dataset.columns if "att" in f]
def f_to_dict(feature):
    indexes = dict( ("avg_over%i_%s" % (i,feature), lambda df: df[:(i+1)].mean()) for i in [0,1,2,3,4,7,14])
    d = {
            "min_%s" % feature:np.min,
            "max_%s" % feature:np.max,
            "mean_%s" % feature :np.mean,
            "std_%s" % feature:np.std
        }
    d.update(indexes)
    dft_list = ["attribute4","attribute5", "attribute6","attribute7","attribute9"]
    if feature in dft_list:
        d["dft_p0_ind%s" % feature] = lambda r : fft_peak(r,p=0,index_no_value=True)
        d["dft_p0_val%s" % feature] = lambda r : fft_peak(r,p=0,index_no_value=False)
        #d["dft_p1_ind%s" % feature] = lambda r : fft_peak(r,p=1,index_no_value=True)
        #d["dft_p1_val%s" % feature] = lambda r : fft_peak(r,p=1,index_no_value=False)
    return d

agg_dict = dict( (f,f_to_dict(f)) for f in features )
#print agg_dict

In [331]:
# bugfix: rolling aggregation after group by does not handle multiple aggregation per column..
# we fix this by flattening the aggregation dict and repeating the data within the dataframe
final_columns = [k for c in agg_dict for k in agg_dict[c]]
input_columns = [c for c in agg_dict for k in agg_dict[c]]
flat_agg_dict = dict( (k,agg_dict[c][k]) for c in agg_dict for k in agg_dict[c])
dup_dataset = pre_dataset[input_columns].sort_index(level="date").sort_index(level="device")
dup_dataset.columns = final_columns

In [332]:
def resample_per_device(df):
    if dataset.index.names == ["device","date"]:
        df = df.swaplevel().sort_index()
    groups = df.groupby(level="device")
    sampled = (
        groups.get_group(g).reset_index(level="device").resample("1D").pad().reset_index()
        for g in groups.groups)
    return pd.concat(sampled).set_index(["date","device"])

In [ ]:
#
# This is where magic happens  !!!
# instead of simply grouping by, we roll over the dataset...
# using the "hacky" flat version, to avoid issues.
# The hack increase the memory consumption quite a lot, but it should still be better than
# explicitely building the windowed lines before aggregating over it. 
#

feature_set = resample_per_device(dup_dataset) \
    .groupby(level="device",as_index=False) \
    .rolling(window=360,min_periods=1) \
    .agg(flat_agg_dict) \
    .reset_index(level=0,drop=True) 
feature_set = post_filter(feature_set).sortlevel(level="device")

In [360]:
# feature filtering
# feature filtering
feat_filtering_thres = 5e-3
try : 
    kept_features = feature_imp[feature_imp.importance > feat_filtering_thres]
    print "threshold : %g, kept: %i features" % (feat_filtering_thres,kept_features.size)
    filtered = feature_set.filter(items=feature_imp.index.sort_values())
    print "filtering devices"
    feature_set = filtered
except:
    print "no feature filtering"

threshold : 0.005, kept: 43 features
filtering devices


In [361]:
#
# use label_window to expand label to neighboring days.
# basically, a mainrtenance x days before failure is still OK
#
label_window = 7

label_set =  resample_per_device(dataset[["failure"]]) \
    .sortlevel(level="date",ascending=False) \
    .groupby(level="device",as_index=False) \
    .rolling(window=label_window, min_periods=1) \
    .sum() \
    .reset_index(level=0,drop=True) 
label_set = filter_devices(label_set).sortlevel(level="device")


In [362]:
negative_subsampling_fraction = 5e-2
idx = pd.IndexSlice

def subsample_negatives(frac,label_set,feature_set):
    set_size = label_set.shape[0]
    pos_size = label_set[label_set["failure"]>0].shape[0]
    print "%i, with %i positives" % (set_size,pos_size)
    sub_label = label_set[label_set["failure"]>0].append(label_set[label_set["failure"]==0].sample(frac=frac))
    sub_feature = feature_set.loc[idx[sub_label_set.index]]
    sub_size = sub_label.shape[0]
    sub_size_0 = sub_feature.shape[0]
    print "new size %i" % sub_size_0 
    return sub_label, sub_feature
sub_label_set , sub_feature_set = subsample_negatives(negative_subsampling_fraction,label_set,feature_set)


131479, with 740 positives
new size 7277


In [364]:
# compute groups (devices), to be used when splitting the training set (train/test)
# This is useful to avoid the bias selection generated by a temporal model
# (Basically, a device used in train cannot also be in test, because its attributes will be very similar)
# this is why the cross-val strategy must split by device, and not merely at random
devices = sub_label_set.index.get_level_values("device")
device_index = dict((device,i) for i,device in enumerate(devices.unique()))
n_dev = len(device_index)
device_groups = np.array(devices.to_series().map(device_index).tolist())

## Run model

In [429]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.preprocessing import Normalizer
from sklearn.metrics import roc_curve, auc
from sklearn.svm import SVC, NuSVC
from sklearn.model_selection import LeavePGroupsOut, GroupShuffleSplit

feature_mat = sub_feature_set.as_matrix()
label_mat = sub_label_set.as_matrix().ravel()

#splitting_strategy = LeavePGroupsOut(n_groups=n_dev/4)
splitting_strategy = GroupShuffleSplit(n_splits=4,test_size=0.33)
                                     
#model = GradientBoostingClassifier()
pca = PCA()#n_components="mle",svd_solver="full")
norm = Normalizer()
#model=GradientBoostingClassifier()
model = RandomForestClassifier()
#model = SVC(probability=True)
pipeline= Pipeline([('normalize', norm),('reduce_dim', pca),("model",model)])
pipeline = pipeline_optimizer._fitted_pipeline
#pipeline= Pipeline([("model",model)])
try:
    # use best parameters if available
    #
    pipeline.set_params(**grid_result.best_params_)
    print "using last optimized model"
except:
    print "no optim result, or bad ones: let's keep the default ones"
    pass
scores = cross_val_score(
    pipeline,
    feature_mat,
    label_mat,
    cv=splitting_strategy,
    groups = device_groups,
    verbose=10,
    scoring="f1",
    n_jobs=6)                          
print "accurracy: %g, std(%g))" % (scores.mean(), scores.std())

no optim result, or bad ones: let's keep the default ones
[CV]  ................................................................
[CV]  ................................................................
[CV]  ................................................................
[CV]  ................................................................
[CV] ................................. , score=0.519481, total=   0.1s
[CV] ................................. , score=0.509240, total=   0.1s
[CV] ................................. , score=0.614876, total=   0.2s
[CV] ................................. , score=0.522727, total=   0.2s
accurracy: 0.541581, std(0.0426086))


[Parallel(n_jobs=6)]: Done   1 tasks      | elapsed:    0.2s
[Parallel(n_jobs=6)]: Batch computation too fast (0.1604s.) Setting batch_size=2.
[Parallel(n_jobs=6)]: Done   2 out of   4 | elapsed:    0.2s remaining:    0.2s
[Parallel(n_jobs=6)]: Done   4 out of   4 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=6)]: Done   4 out of   4 | elapsed:    0.2s finished


In [430]:
pipeline.get_params()

{'binarizer': Binarizer(copy=True, threshold=0.3),
 'binarizer__copy': True,
 'binarizer__threshold': 0.3,
 'gaussiannb': GaussianNB(priors=None),
 'gaussiannb__priors': None,
 'robustscaler': RobustScaler(copy=True, quantile_range=(25.0, 75.0), with_centering=True,
        with_scaling=True),
 'robustscaler__copy': True,
 'robustscaler__quantile_range': (25.0, 75.0),
 'robustscaler__with_centering': True,
 'robustscaler__with_scaling': True,
 'selectfwe': SelectFwe(alpha=0.05, score_func=<function f_classif at 0x7fefd403c230>),
 'selectfwe__alpha': 0.05,
 'selectfwe__score_func': <function sklearn.feature_selection.univariate_selection.f_classif>,
 'steps': [('robustscaler',
   RobustScaler(copy=True, quantile_range=(25.0, 75.0), with_centering=True,
          with_scaling=True)),
  ('binarizer', Binarizer(copy=True, threshold=0.3)),
  ('selectfwe',
   SelectFwe(alpha=0.05, score_func=<function f_classif at 0x7fefd403c230>)),
  ('gaussiannb', GaussianNB(priors=None))]}

### Eval Model

In [431]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, accuracy_score,precision_recall_curve, auc

X_train, X_test, Y_train, Y_test = train_test_split(feature_mat,label_mat,test_size=0.3)
# calculate the fpr and tpr for all thresholds of the classification

fitted = pipeline.fit(X_train,Y_train)
probs = fitted.predict_proba(X_test)
preds = probs[:,1]
preds_train = fitted.predict_proba(X_train)[:,1]
fpr, tpr, threshold = roc_curve(Y_test, preds)
fpr_train, tpr_train, threshold_train = roc_curve(Y_train, preds_train)
roc_auc = auc(fpr, tpr)
roc_auc_train = auc(fpr_train, tpr_train)
precision, recall, ths = precision_recall_curve(Y_test, preds)
precision_train, recall_train, ths_train = precision_recall_curve(Y_train, preds_train)

In [432]:
from bokeh.models.ranges import Range1d
#print "auc: %.2g, on train: %.2g" %(roc_auc, roc_auc_train)
roc_df = pd.DataFrame({"fpr":fpr,"tpr":tpr}).set_index("fpr")
pr_df = pd.DataFrame({"precision": precision, "recall":recall}).set_index("recall")
roc_df["diag"] = roc_df.index
pr_df["random"] = pr_df.precision.iloc[0]

# roc curve
roc_f = figure(width=400,height=400,title="roc, auc: %.2g, on train: %.2g"  %(roc_auc, roc_auc_train) )
roc_f.xaxis.axis_label = "tpr"
auc_range= Range1d(0,1)
roc_f.x_range = auc_range 
roc_f.y_range = auc_range 
roc_f.yaxis.axis_label = "fpr"
roc_f.cross(fpr,tpr,size=5)
roc_f.line(fpr,tpr,legend="roc")
roc_f.circle(fpr_train,tpr_train,size=5,color="red", line_width=1)
roc_f.line(fpr_train,tpr_train,color="red",legend="roc on train")
roc_f.line([0,1],[0,1], color="grey")

# pr curve
pr_f = figure(width=400,height=400,title="PR curve")
pr_f.xaxis.axis_label = "recall"
pr_f.yaxis.axis_label = "precision"
pr_f.cross(recall,precision,size=5)
pr_f.line(recall,precision,legend="PR")
pr_f.circle(recall_train,precision_train,size=5,color="red", line_width=1)
pr_f.line(recall_train,precision_train,color="red",legend="PR on train")

show(row(
    pr_f,
    roc_f
))

### Feature Importance

In [370]:
feature_imp = pd.DataFrame({"importance":model.feature_importances_}).set_index(feature_set.columns)
feature_imp.sort_values(by="importance",ascending=False)

,importance
avg_over2_attribute7,0.075044
avg_over2_attribute2,0.052664
avg_over1_dt2_attribute2,0.047126
avg_over1_attribute7,0.041062
avg_over0_dt2_attribute2,0.040053
avg_over1_attribute5,0.037641
avg_over14_attribute9,0.037396
avg_over1_attribute4,0.035768
avg_over2_attribute5,0.035039
avg_over0_attribute7,0.034961


### Hyperparameter optimisation




In [371]:
# model : GradientBoostingClassifier, parameters: 
#loss : {‘deviance’, ‘exponential’},
#learning_rate : float, optional (default=0.1)
#n_estimators : int (default=100)
#max_depth : integer, optional (default=3)
#min_samples_split : int, float, optional (default=2)
grids=dict()
XDB_param_grid = {
    #"model__loss":  ["deviance", 'exponential'],
    "model__learning_rate" : [1e-3,0.01, 0.1],
    "model__n_estimators" : [10, 50, 100, 150],
    "model__max_depth" : [5,10,15],
    "model__min_samples_split" : [5,10,20]
}
grids[GradientBoostingClassifier] = XDB_param_grid

In [372]:
# C :penalty
# kernel : ‘linear’, ‘poly’, ‘rbf’, ‘sigmoid’, ‘precomputed’ 
SVC_param_grid = {
   'model__C': [1e-7,1e-6,1e-5,0.1],
    "model__kernel": ["rbf","linear"],
    #"model_degree" : [1,3,5], # polynomial degrees
    "model__gamma" : ["auto"], # kernel coef (rbf)
    #"coef0" # for poly, signmoid
    "model__tol" : [1e-7,1e-6,1e-5, 1e-4,1e-3]
}
grids[SVC] = SVC_param_grid

In [373]:
# model : RandomForestClassifier, parameters: 
# n_estimators : int (default=100)
# criterion : "gini","entropy"
# max_features : auto , fraction
# max_depth : integer, optional (default=3)
# min_samples_split : int, float, optional (default=2)

RF_param_grid = {
    "model__criterion":  ["entropy"],
    "model__n_estimators" : [100,200,300,400],
    "model__max_features" : ["auto"],
    "model__max_depth" : [20,30,40],
    "model__min_samples_split" : [10,20,30]
}
grids[RandomForestClassifier] = RF_param_grid

In [374]:
m  = type(dict(pipeline.steps)["model"])
param_grid=grids[m]

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold

splits = splitting_strategy.split(feature_mat,label_mat,device_groups)
#kfold = StratifiedKFold(n_splits=6, shuffle=True)
grid_search = GridSearchCV(
    pipeline, 
    param_grid,
    scoring="f1",
    n_jobs=-1,
    verbose=1,
    cv=splits
)
grid_result = grid_search.fit(feature_mat,label_mat)


# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
#for mean, stdev, param in zip(means, stds, params):
#    print("%f (%f) with: %r" % (mean, stdev, param))

Fitting 4 folds for each of 36 candidates, totalling 144 fits


[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 144 out of 144 | elapsed:  5.5min finished


Best: 0.380537 using {'model__n_estimators': 100, 'model__max_features': 'auto', 'model__max_depth': 40, 'model__min_samples_split': 30, 'model__criterion': 'entropy'}


In [ ]:
pipeline.get_params()

In [485]:
from tpot import TPOTClassifier
pipeline_optimizer = TPOTClassifier(
    generations=10, # the more generatiion, the more optimized you get
    population_size=200,
    num_cv_folds=4,
    scoring="f1",
    random_state=42,
    verbosity=3)

pipeline_optimizer.fit(feature_mat, label_mat)
print pipeline_optimizer.score(feature_mat, label_mat)
pipeline_optimizer.export('tpot_longrun_exported_pipeline.py')

Optimization Progress:   0%|          | 6/2200 [04:23<22:12:02, 36.43s/pipeline]n Progress:   0%|          | 1/2200 [00:40<24:32:02, 40.16s/pipeline]          Optimization Progress:   0%|          | 5/2200 [04:23<31:41:28, 51.98s/pipeline]

Timeout during evaluation of pipeline #5. Skipping to the next pipeline.


Optimization Progress:   1%|          | 17/2200 [08:09<4:17:38,  7.08s/pipeline] ptimization Progress:   0%|          | 8/2200 [06:23<32:43:34, 53.75s/pipeline]          Optimization Progress:   1%|          | 18/2200 [08:09<14:28:24, 23.88s/pipeline]

Timeout during evaluation of pipeline #18. Skipping to the next pipeline.


Optimization Progress:   1%|          | 23/2200 [13:02<30:10:12, 49.89s/pipeline]Optimization Progress:   1%|          | 20/2200 [09:42<18:55:08, 31.24s/pipeline]          Optimization Progress:   1%|          | 24/2200 [13:02<36:25:49, 60.27s/pipeline]

...

Generation 10 - Current Pareto front scores:
1	0.556803458605	BernoulliNB(input_matrix, 0.83999999999999997, 6.0)
2	0.564612431998	BernoulliNB(DecisionTreeClassifier(input_matrix), 0.10000000000000001, 0.070000000000000007)
3	0.576280240377	BernoulliNB(DecisionTreeClassifier(LogisticRegression(input_matrix, 0.78000000000000003, 25, False)), 0.20000000000000001, 0.070000000000000007)



Optimization Progress:  97%|█████████▋| 2143/2200 [4:28:51<14:14, 15.00s/pipeline]Optimization Progress:  91%|█████████ | 2003/2200 [4:14:05<04:22,  1.33s/pipeline]Optimization Progress:  91%|█████████▏| 2012/2200 [4:14:24<01:21,  2.31pipeline/s]Optimization Progress:  94%|█████████▎| 2057/2200 [4:14:44<00:12, 11.47pipeline/s]Optimization Progress:  94%|█████████▍| 2065/2200 [4:15:04<03:48,  1.69s/pipeline]Optimization Progress:  96%|█████████▋| 2121/2200 [4:17:04<00:15,  5.19pipeline/s]          Optimization Progress:  97%|█████████▋| 2144/2200 [4:28:51<2:19:15, 149.20s/pipeline]

Timeout during evaluation of pipeline #2144. Skipping to the next pipeline.


                                                                                    ]Optimization Progress:  98%|█████████▊| 2147/2200 [4:28:51<1:04:40, 73.22s/pipeline] Optimization Progress:  98%|█████████▊| 2159/2200 [4:29:06<05:55,  8.67s/pipeline]

0.679319371728


In [488]:
pipeline_optimizer.export("device_time_export.py")
#with file("device_time-export.feat","w+ as feature_mat

In [486]:
model_name = "device_time"

from sklearn.externals import joblib
import pickle
joblib.dump(pipeline_optimizer._fitted_pipeline, 'model_%s.pkl' % model_name) 
features = feature_set.columns
with open('model_%s.feat' % model_name, "w+") as f: 
    pickle.dump(features,f)

PicklingError: Can't pickle <function <lambda> at 0x7fefd9f3b578>: it's not found as __main__.<lambda>

In [397]:
# mixing model_per_device and model_per_device_and_time
model_name = "device"

from sklearn.externals import joblib
import pickle

device_model = joblib.load("model.pkl")
device_model.named_steps
with open('model_%s.feat' % model_name, "r") as f: 
    device_model_features = pickle.load(f)

In [462]:
device_mat = sub_feature_set[device_model_features]
predicted_device = device_model.predict_proba(device_mat)

In [481]:
device_prediction = predicted_device[:,1]

device_time_prediction = device_time_model.predict_proba(feature_mat)[:,1]
#device_time_predictions = fitted.predict_proba(feature_mat)[:,1]
# basic (two models) vote
confidence_0 = np.abs(device_prediction -0.5)
confidence_1 = np.abs(device_time_prediction -0.5)
vote_0 = confidence_0 > confidence_1
vote_1 =  np.invert(vote_0)

#prediction = vote_0*device_prediction +vote_1*device_time_predictions
prediction= device_time_predictions*device_prediction

In [482]:
total_prec, total_recall, ths = precision_recall_curve(label_mat, prediction)
dev_prec, dev_recall, ths = precision_recall_curve(label_mat, device_prediction)
dev_t_prec, dev_t_recall, ths = precision_recall_curve(label_mat, device_time_prediction)

In [483]:
# pr curve
tpr_f = figure(width=400,height=400,title="PR curve")
tpr_f.xaxis.axis_label = "recall"
tpr_f.yaxis.axis_label = "precision"

tpr_f.cross(dev_recall,dev_prec,size=5,color="green")
tpr_f.line(dev_recall,dev_prec,legend="dev PR",color="green")

tpr_f.cross(dev_t_recall,dev_t_prec,size=5,color="red")
tpr_f.line(dev_t_recall,dev_t_prec,legend="dev time PR",color="red")

tpr_f.cross(total_recall,total_prec,size=5)
tpr_f.line(total_recall,total_prec,legend="PR")

show(tpr_f)